# Credmark Modeling Framework Example for Jupyter notebook
## Ledger Model

version: 2023.4.15

## Initialize

<div class="alert alert-block alert-info">
    <b>Note:</b> Change to a web3 provider you have in `params`/`chain_to_provider_url` below.
</div>

In [ ]:
%reload_ext credmark.cmf.ipython

param = {
    'chain_id': 1,
    'block_number': None,         
    'chain_to_provider_url': {'1': 'https://mainnet.infura.io/v3/... or https://eth-mainnet.g.alchemy.com/'},
    'api_url': None,
    'use_local_models': None,
    'register_utility_global': True}

context, model_loader = %cmf param

## Example

### Contract - Uni V2

In [ ]:
contract = Contract(address='0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852')
print(contract.functions.name().call(), contract.functions.symbol().call())

for addr in [contract.functions.token0().call(), contract.functions.token1().call()]:
    print(Token(address=addr).symbol)

### Ledger Examples

- Get the max value group by `to`

In [ ]:
contract = Contract(address='0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852')

with contract.ledger.functions.Transfer as q:
    ret = (q.select(
        aggregates=[(q.VALUE.max_(), 'max_value')],
        group_by=[q.TO,q.TXN_BLOCK_NUMBER],
        order_by=q.field('max_value').dquote().desc(),
        where=q.TXN_BLOCK_NUMBER.gt(context.block_number - 2000000),
        limit=10
    ).to_dataframe())
display(ret)

- Get max amount0Out by `to`.

In [ ]:
with contract.ledger.events.Swap as q:
    ret = (q.select(
        aggregates=[(q.AMOUNT0OUT.max_(), 'max_value')],
        group_by=[q.TO],
        order_by=q.field('max_value').dquote().desc(),
        where=q.EVT_BLOCK_NUMBER.gt(context.block_number - 2000000),
        limit=5)
    .to_dataframe())
display(ret)

- Get max amount0Out and max amount1Out

In [ ]:
contract = Contract(address='0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852')
with contract.ledger.events.Swap as q:
    ret = (q.select(
        aggregates=[(q.AMOUNT0OUT.max_(), 'max_amount0Out'), (q.AMOUNT1OUT.max_(), 'max_amount1Out')]
    ).to_dataframe())
display(ret)

- Get the transaction hash containing the max amount0Out.

In [ ]:
contract = Contract(address='0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852')
with contract.ledger.events.Swap as q:
    ret2 = (q.select(
        aggregates=[(q.AMOUNT0OUT.max_(), 'max_amount0Out')],
        group_by=[q.EVT_HASH],
        having=q.AMOUNT0OUT.max_().ge(ret.max_amount0Out[0]),
    ).to_dataframe())
display(ret2)

- Get the transaction hash containing the max amount0Out or max amount1Out

For any column in `having` needs to be in `aggregates`.

In [ ]:
contract = Contract(address='0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852')
with contract.ledger.events.Swap as q:
    ret3 = (q.select(
        aggregates=[(q.AMOUNT0OUT.max_(), 'max_amount0Out'),
                    (q.AMOUNT1OUT.max_(), 'max_amount1Out')],
        group_by=[q.EVT_BLOCK_NUMBER, q.EVT_HASH],
        having=q.AMOUNT0OUT.max_().ge(ret.max_amount0Out[0]).or_(
            q.AMOUNT1OUT.max_().ge(ret.max_amount1Out[0])),
        order_by=q.EVT_BLOCK_NUMBER
    ).to_dataframe())

display(ret3)

In [ ]:
# Below web3 may not work with some node servers
if False:
    swap_events = contract.events.Swap.createFilter(
        fromBlock=0,
        toBlock=context.block_number
    ).get_all_entries()


In [ ]:
pd.DataFrame(contract.fetch_events(contract.events.Swap, from_block=block_number-1000, to_block=block_number-990))